In this notebook, I am going to use glm and lmer R functions to fit some multi-level linear regression models to some synthetic data. Specfically, I will focus on showing how different statistics can be used to select the best structure for the multi-level model.


First, let's add the needed libraries:

In [11]:
country.code <- 'ca'  # use yours
url.pattern <- 'https://'  # use http if you want
repo.data.frame <- subset(getCRANmirrors(), CountryCode == country.code & grepl(url.pattern, URL))
options(repos = repo.data.frame$URL)

In [12]:
install.packages("arm")


Warning message:
: package 'arm' is in use and will not be installed

In [13]:
library(arm)  # convenience functions for regression in R


The synthetic data is a simulation of a behavioral experiment. In this experiment we have two subjects. For each subject we have 10 trials and within each trial the test parameter has changed randomly. There are two conditions in the test. We would like to fit a linear regression to this dataset. 
The synthetic data consists of linear model in which the coefficient and the intercept change for each subject and each condition. Gaussian noise is added at the end to the generated dataset.


We first set the parameters of the simulated model:

In [14]:
intercept.sbjt1.cond1 <- 1
intercept.sbjt1.cond2 <- .5
intercept.sbjt2.cond1 <- 1
intercept.sbjt2.cond2 <- .5

coeff.sbjt1.cond1 <- 1
coeff.sbjt1.cond2 <- .7
coeff.sbjt2.cond1 <- 1
coeff.sbjt2.cond2 <- .7

Here are the main simulations with the specfied parameters:

In [15]:
X.sbjt1.cond1 <- runif(10, min=-1, max=1)
X.sbjt1.cond2 <- runif(10, min=-1, max=1)
X.sbjt2.cond1 <- runif(10, min=-1, max=1)
X.sbjt2.cond2 <- runif(10, min=-1, max=1)

y.sbjt1.cond1 <- coeff.sbjt1.cond1 *  X.sbjt1.cond1 + intercept.sbjt1.cond1
y.sbjt1.cond2 <- coeff.sbjt1.cond2 *  X.sbjt1.cond2 + intercept.sbjt1.cond2
y.sbjt2.cond1 <- coeff.sbjt2.cond1 *  X.sbjt2.cond1 + intercept.sbjt2.cond1
y.sbjt2.cond2 <- coeff.sbjt2.cond2 *  X.sbjt2.cond2 + intercept.sbjt2.cond2


In the next step, we pool the subjects' data to prepare it for the linear model:


In [16]:
X.cond1 <- c(X.sbjt1.cond1,X.sbjt2.cond1)
X.cond2 <- c(X.sbjt1.cond2,X.sbjt2.cond2)
y.cond1 <- c(y.sbjt1.cond1,y.sbjt2.cond1)
y.cond2 <- c(y.sbjt1.cond2,y.sbjt2.cond2)
s.cond1 <- c(matrix(0,1,10),matrix(1,1,10))
s.cond2 <- c(matrix(0,1,10),matrix(1,1,10))

X1 <- array(X.cond1, dim=c(20,1))
Y1 <- array(y.cond1, dim=c(20,1))
C1 <- array(s.cond1, dim=c(20,1))
X2 <- array(X.cond2, dim=c(20,1))
Y2 <- array(y.cond2, dim=c(20,1))
C2 <- array(s.cond2, dim=c(20,1))

df1 <- data.frame(X1,C1,Y1)
df2 <- data.frame(X2,C2,Y2)


df1 and df2 contain all the simulated data for the first and the second subjects, respectively. 


Using the glm toolbox, we first fit an ordinary linear regression model to the data. In this model, the differences between the two subjects will be ignored, and the model will be identified as if all the samples are exchangeable:


In [17]:
# ordinary linear regression with glm library
MLexamp1.1 <- glm(Y1 ~ X1, data = df1)
MLexamp2.1 <- glm(Y2 ~ X2, data = df2)

cat("cond_1 Fixed-effect AIC = ", AIC(MLexamp1.1),"\n")
cat("cond_2 Fixed-effect AIC = ",AIC(MLexamp2.1),"\n")

cond_1 Fixed-effect AIC =  -1381.947 
cond_2 Fixed-effect AIC =  -1376.633 


The goodness of fit is being assessed by the AIC metric at the end.


Now, we use the lmer function to fit a multi-level linear regression model to the same data:  